# Comparative Analysis of TMP Databases

**Project:** Digital Teotihuacan Mapping Project (TMP) - Phase 1

**Objective:** This notebook synthesizes the results from the entire profiling pipeline to conduct a comparative analysis of the four legacy databases and the two wide-format benchmark databases. Its primary goal is to use quantitative data to compare these database architectures on three key axes: **Structural Complexity**, **Resource Usage**, and **Query Performance**. 

The findings from this notebook will directly inform the final recommendation for the Phase 2 unified database architecture.

--- 
## 1. Setup and Configuration

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path
from IPython.display import display, Markdown
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
import os

# --- Path Definitions ---
# Use more robust path construction based on notebook file location
try:
    # Try to get notebook's directory (works in most Jupyter environments)
    NOTEBOOK_DIR = Path(os.getcwd())
    # Find the project root by looking for specific markers
    current_path = NOTEBOOK_DIR
    while current_path != current_path.parent:
        if (current_path / "TASKS.md").exists() or (current_path / "pyproject.toml").exists():
            PROJECT_ROOT = current_path
            break
        current_path = current_path.parent
    else:
        # Fallback: assume standard structure
        PROJECT_ROOT = NOTEBOOK_DIR.parent.parent.parent.parent
    
    REPORTS_DIR = PROJECT_ROOT / "phases" / "01_LegacyDB" / "outputs" / "reports"
except Exception as e:
    print(f"Warning: Path detection failed ({e}). Using fallback path construction.")
    # Fallback to original approach
    PROJECT_ROOT = Path.cwd().parent.parent
    REPORTS_DIR = PROJECT_ROOT / "outputs" / "reports"

# --- Styling and Display Options ---
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

def display_header(title):
    display(Markdown(f'### {title}'))

print("✅ Setup complete.")
print(f"Project Root: {PROJECT_ROOT}")
print(f"Reports Directory: {REPORTS_DIR}")
print(f"Reports Directory exists: {REPORTS_DIR.exists()}")

--- 
## 2. Data Loading
We load two key outputs from the `04_run_comparison.py` script:
1. `comparison_matrix.csv`: High-level summary metrics.
2. `report_performance_summary_detailed.csv`: The enriched, long-form performance data with calculated comparative metrics.

In [ ]:
matrix_path = REPORTS_DIR / 'comparison_matrix.csv'
perf_path = REPORTS_DIR / 'report_performance_summary_detailed.csv'

# Validate file existence with detailed error reporting
missing_files = []
if not matrix_path.exists():
    missing_files.append(str(matrix_path))
if not perf_path.exists():
    missing_files.append(str(perf_path))

if missing_files:
    print(f"Directory contents: {list(REPORTS_DIR.glob('*')) if REPORTS_DIR.exists() else 'Directory does not exist'}")
    raise FileNotFoundError(f"Critical Error: The following report files were not found:\n" + 
                           "\n".join(f"  - {f}" for f in missing_files) + 
                           f"\n\nSearched in: {REPORTS_DIR}\n" +
                           "Please run the 04_run_comparison.py script first to generate these files.")

try:
    # Load the matrix and transpose it so databases are rows
    comparison_df = pd.read_csv(matrix_path, index_col=0).T.reset_index().rename(columns={'index': 'Database'})
    print(f"✅ Successfully loaded comparison matrix: {comparison_df.shape[0]} databases, {comparison_df.shape[1]} metrics")
    
    # Load the detailed performance data
    perf_summary_df = pd.read_csv(perf_path)
    print(f"✅ Successfully loaded performance data: {perf_summary_df.shape[0]} records")
    
except Exception as e:
    print(f"Error loading data files: {e}")
    raise

print("\nLoaded Comparison Matrix:")
display(comparison_df)

print("\nLoaded Detailed Performance Summary Data:")
display(perf_summary_df.head())
print(f"\nPerformance data columns: {list(perf_summary_df.columns)}")
print(f"Unique databases in performance data: {perf_summary_df['database'].unique() if 'database' in perf_summary_df.columns else 'No database column found'}")

--- 
## 3. High-Level Comparison Matrix
A styled view of the main comparison matrix. Color gradients highlight high/low values for each metric, providing an at-a-glance summary.
- <span style='color: #440154;'>**Purple/Dark**</span>: Higher values
- <span style='color: #fde725;'>**Yellow/Light**</span>: Lower values

In [ ]:
display_header("Styled Comparison Matrix")

styled_df = comparison_df.style.background_gradient(cmap='viridis', axis=0)\
    .set_caption("Comparative Database Metrics")\
    .format('{:.2f}', subset=pd.IndexSlice[:, ['Database Size (MB)', 'JDI (Join Dependency Index)', 'NF (Normalization Factor)']])\
    .format('{:,.0f}', subset=pd.IndexSlice[:, ['Table Count', 'View Count', 'Total Estimated Rows', 'Total Index Count', 'LIF (Logical Interop. Factor)']])

display(styled_df)

--- 
## 4. Structural Complexity Analysis
This section focuses on the metrics that quantify the relational complexity and degree of normalization of the legacy schemas.

In [ ]:
display_header("Schema Complexity Metrics (Legacy Databases)")

complexity_metrics = [
    'Database', 'Table Count', 
    'JDI (Join Dependency Index)', 
    'LIF (Logical Interop. Factor)',
    'NF (Normalization Factor)'
]
# Filter for legacy DBs only, as these metrics don't apply to the single-table benchmarks
legacy_df = comparison_df[~comparison_df['Database'].str.contains('benchmark')]
display(legacy_df[complexity_metrics])

# --- Advanced Visualization: Complexity Radar Plot ---
radar_metrics = ['Table Count', 'JDI (Join Dependency Index)', 'NF (Normalization Factor)']
radar_df = legacy_df[['Database'] + radar_metrics].copy()

# Normalize metrics to a 0-1 scale for fair comparison on the radar plot
scaler = MinMaxScaler()
radar_df[radar_metrics] = scaler.fit_transform(radar_df[radar_metrics])

fig = go.Figure()

for index, row in radar_df.iterrows():
    fig.add_trace(go.Scatterpolar(
        r=row[radar_metrics].values,
        theta=radar_metrics,
        fill='toself',
        name=row['Database']
    ))

fig.update_layout(
  polar=dict(
    radialaxis=dict(
      visible=True,
      range=[0, 1]
    )),
  showlegend=True,
  title='Normalized Complexity Profile of Legacy Databases'
)

fig.show()

--- 
## 5. Query Performance Deep Dive
This is the most critical comparison. It directly measures the analytical query performance of the legacy normalized schemas against the denormalized wide-format benchmark schemas using the pre-calculated metrics from the `04_run_comparison.py` script.

In [ ]:
display_header("Schema Efficiency Factor by Query Category")

if not perf_summary_df.empty:
    # First, display the tabular data (NOT logarithmic)
    print("📊 Schema Efficiency Factor Data Table (Raw Values):")
    print("Note: Values > 1.0 indicate the database is slower than the benchmark baseline")
    
    # Create a pivot table for better readability
    if 'schema_efficiency_factor' in perf_summary_df.columns:
        efficiency_table = perf_summary_df.pivot_table(
            index='database', 
            columns='category', 
            values='schema_efficiency_factor',
            aggfunc='mean'
        ).round(2)
        
        # Add a summary column showing average across categories
        efficiency_table['Average'] = efficiency_table.mean(axis=1).round(2)
        
        # Style the table to highlight high values
        styled_efficiency = efficiency_table.style.background_gradient(cmap='Reds', axis=None)\
            .set_caption("Schema Efficiency Factors (Lower is Better - Benchmark = 1.0)")\
            .format('{:.2f}')
        
        display(styled_efficiency)
        
        # Also show summary statistics
        print(f"\n📈 Summary Statistics:")
        print(f"Worst performing database (highest average): {efficiency_table['Average'].idxmax()} ({efficiency_table['Average'].max():.2f}x slower)")
        print(f"Best performing database (lowest average): {efficiency_table['Average'].idxmin()} ({efficiency_table['Average'].min():.2f}x)")
    else:
        print("⚠️ 'schema_efficiency_factor' column not found in performance data")
        display(perf_summary_df)
    
    print("\n" + "="*50)
    print("📊 Schema Efficiency Factor Chart (Log Scale):")
    
    # Then display the chart with log scale
    fig = px.bar(perf_summary_df,
                 x='database', 
                 y='schema_efficiency_factor', 
                 color='category', 
                 barmode='group',
                 title='Schema Efficiency Factor (Lower is Better)',
                 labels={'schema_efficiency_factor': 'Efficiency Factor (Log Scale)', 'database': 'Database'},
                 category_orders={'category': ['baseline', 'join_performance', 'complex_filtering']})
    
    fig.update_yaxes(type="log") # Use a log scale as differences can be huge
    fig.add_hline(y=1.0, line_dash="dot", annotation_text="Benchmark Baseline", annotation_position="bottom right")
    fig.update_layout(height=600)
    fig.show()
else:
    print("No performance data to plot.")

In [ ]:
display_header("Performance Improvement vs. Best Benchmark")

if not perf_summary_df.empty:
    # First, display the tabular data
    print("📊 Performance Improvement Data Table:")
    print("Note: Positive values indicate how much faster the benchmark is compared to legacy databases")
    
    # Filter out the baseline databases themselves for cleaner analysis
    improvement_df = perf_summary_df[~perf_summary_df['database'].str.contains('benchmark')].copy()
    
    if not improvement_df.empty and 'performance_improvement_factor' in improvement_df.columns:
        # Create a summary table showing improvement factors
        improvement_summary = improvement_df.groupby(['database', 'category'])['performance_improvement_factor'].agg(['mean', 'min', 'max']).round(1)
        improvement_summary.columns = ['Avg_Improvement_%', 'Min_Improvement_%', 'Max_Improvement_%']
        
        # Reset index to make it more readable
        improvement_summary = improvement_summary.reset_index()
        improvement_pivot = improvement_summary.pivot(index='database', columns='category', values='Avg_Improvement_%').round(1)
        
        # Style the table
        styled_improvement = improvement_pivot.style.background_gradient(cmap='Greens', axis=None)\
            .set_caption("Average Performance Improvement (% faster than benchmark)")\
            .format('{:.1f}%')
        
        display(styled_improvement)
        
        # Show detailed breakdown
        print(f"\n📋 Detailed Performance Improvement Breakdown:")
        detailed_table = improvement_df[['database', 'category', 'query_id', 'performance_improvement_factor']].copy()
        detailed_table['performance_improvement_factor'] = detailed_table['performance_improvement_factor'].round(1)
        detailed_table = detailed_table.rename(columns={
            'performance_improvement_factor': 'Improvement_%'
        })
        
        # Sort by improvement factor for better readability
        detailed_table = detailed_table.sort_values(['database', 'category', 'Improvement_%'], ascending=[True, True, False])
        display(detailed_table)
        
        # Summary statistics
        print(f"\n📈 Key Insights:")
        best_db = improvement_pivot.mean(axis=1).idxmin()
        worst_db = improvement_pivot.mean(axis=1).idxmax()
        print(f"Most consistent performer: {best_db} (avg {improvement_pivot.mean(axis=1)[best_db]:.1f}% improvement over benchmark)")
        print(f"Least consistent performer: {worst_db} (avg {improvement_pivot.mean(axis=1)[worst_db]:.1f}% improvement over benchmark)")
        
    else:
        print("⚠️ 'performance_improvement_factor' column not found or no legacy database data available")
        if not improvement_df.empty:
            display(improvement_df)
    
    print("\n" + "="*50)
    print("📊 Performance Improvement Chart:")
    
    # Then display the chart
    fig = px.bar(improvement_df.sort_values('performance_improvement_factor') if not improvement_df.empty else improvement_df,
                 x='query_id', 
                 y='performance_improvement_factor', 
                 color='database', 
                 facet_row='category',
                 barmode='group',
                 title='Performance Improvement of Benchmark Schemas vs. Legacy Schemas',
                 labels={'performance_improvement_factor': '% Improvement vs. Benchmark', 'query_id': 'Query ID'})
    
    fig.update_layout(height=800)
    fig.show()
else:
    print("Could not generate performance improvement plot.")

--- 
## 6. Qualitative Architectural Trade-offs
The quantitative data above supports a qualitative assessment of the architectural trade-offs between the legacy design and the proposed wide-format design.

| Feature                  | Legacy Normalized (e.g., DF9)      | Proposed Wide-Format (Benchmark)   | Justification Based on Data                                                                      |
| :----------------------- | :--------------------------------- | :--------------------------------- | :----------------------------------------------------------------------------------------------- |
| **Query Performance** | `Low`                              | `High`                             | The 'Schema Efficiency Factor' chart shows legacy databases are multiple times slower.             |
| **Storage Cost** | `Low`                              | `High`                             | `comparison_matrix.csv` shows benchmark DBs are larger due to data duplication.                |
| **Schema Complexity** | `High` (High JDI/NF, Many Tables)  | `Very Low` (1 Table)               | The complexity radar plot visually confirms the high complexity scores of the legacy schemas.    |
| **Data Redundancy** | `Low` (Normalized)                 | `High` (Denormalized)              | This is the inherent trade-off of the wide-format design; we trade storage for speed.            |
| **Ease of Use for BI/GIS** | `Low` (Requires complex joins)     | `High` (Single table source)       | A single flat table is trivial to connect to tools like QGIS, Tableau, or Power BI.              |

---
## 7. Final Analyst Summary & Recommendation

**Instructions:** Based on the comparative analysis, synthesize the findings and provide a formal recommendation for the Phase 2 unified database architecture. This summary will be a primary input for the final white paper.

### Overarching Conclusion:
* *Start with a concise, definitive statement. Example: "The comparative analysis demonstrates conclusively that the highly normalized structure of the legacy databases, particularly `tmp_df9`, is quantitatively inferior for the project's analytical objectives compared to a denormalized, wide-format architecture."*

### Justification from Evidence:
1.  **On Performance:**
    * *Quantify the performance difference. Reference the 'Schema Efficiency Factor' chart directly. Example: "As shown in the efficiency factor plot, the legacy schemas are between 5x and 50x slower for join-heavy queries than the wide-format benchmarks. This performance gap makes interactive analysis on the normalized schemas untenable."*
2.  **On Complexity:**
    * *Reference the complexity metrics and the radar plot. Example: "The legacy schemas exhibit high JDI and NF scores, indicative of significant relational complexity that increases the cognitive load for analysts and the technical barrier for connecting to BI and GIS tools. The radar plot clearly visualizes `tmp_df9` as the most complex outlier."*
3.  **On The Cost/Benefit Trade-off:**
    * *Acknowledge the trade-offs identified in the qualitative table. Example: "While the wide-format approach increases storage costs due to data redundancy, this trade-off is strategically acceptable. The cost of storage is minimal compared to the significant gains in query performance and the drastic reduction in development time and analytical friction for end-users."*

### Formal Recommendation:
* *State the final recommendation clearly and unambiguously.*
* **Recommended Architecture:** "It is the formal recommendation of this analysis that Phase 2 of the Digital TMP project proceeds with the development of a single, denormalized, wide-format primary analytical table. This table should be based on the schema of the `tmp_benchmark_wide_text_nulls` database, as it provides the best balance of performance and human-readability."*
* **Next Steps:** "The next step should be to finalize the schema of this wide-format table, including data type assignments and column naming conventions, and to proceed with the development of the full ETL pipeline in Phase 2 to migrate all legacy data into this new structure."*